In [26]:
import sys
!{sys.executable} -m pip install nba_api
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install langchain
!{sys.executable} -m pip install yt-dlp
!{sys.executable} -m pip install openai-whisper
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install tiktoken

3417.91s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


3424.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


3430.36s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


3436.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


3442.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


3448.29s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


3454.28s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


3460.41s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.3/698.3 kB 6.3 MB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 26.6 MB/s eta 0:00:0031m31.6 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lxml: filename=lxml-4.9.2-cp310-cp310-macosx_12_0_arm64.whl size=1676870 sha256=bb80aa157ed2d1643295acfe6292a5e6d0f058b03806fe08b9bbb02341002012
  Stored in directory: /Users/alessiofanelli/Library/Caches/pip/wheels/5a/51/0e/95b4a6ddee4a616530c36aeb03dafb5e04183756d9973a7d5d
Successfully built lxml


In [5]:
import yt_dlp

URLS = ['https://www.youtube.com/watch?v=57OU18cogJI']

ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }],
    'outtmpl': './tmp/foo_%(title)s-%(id)s.%(ext)s'
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(URLS)

[youtube] Extracting URL: https://www.youtube.com/watch?v=57OU18cogJI
[youtube] 57OU18cogJI: Downloading webpage
[youtube] 57OU18cogJI: Downloading android player API JSON
[info] 57OU18cogJI: Downloading 1 format(s): 140
[download] Destination: ./tmp/foo_StrictlyVC in conversation with Sam Altman, part one-57OU18cogJI.m4a
[download] 100% of   19.03MiB in 00:00:00 at 22.37MiB/s  
[FixupM4a] Correcting container of "./tmp/foo_StrictlyVC in conversation with Sam Altman, part one-57OU18cogJI.m4a"
[ExtractAudio] Not converting audio ./tmp/foo_StrictlyVC in conversation with Sam Altman, part one-57OU18cogJI.m4a; file is already in target format m4a


In [8]:
import whisper
import os

model = whisper.load_model("base")

podcasts_to_analyze = []

for file in os.listdir("./tmp"):
    # Skip if the file is not a video or audio file
    if not file.endswith(".m4a"):
        continue
    
    file_path = os.path.join("./tmp", file)
    result = model.transcribe(file_path)
    podcasts_to_analyze.append(result["text"])

/opt/homebrew/lib/python3.10/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [36]:
print(podcasts_to_analyze)

[" So appreciate it. Thanks for having me. So Sam, you also very nicely came to an event that happened to be right here three years ago. Yeah, I remember. What's been happening? Um, open the eye mostly. Yeah, it's just taken up a lot of my time, but it's super great. I think we're doing a lot of stuff we're really proud of. I mean, I'm half kidding, obviously. But you know, you've been at the center of the startup conversation for almost 10 years since you had taken over as the president of a way. Combinator. Wow, that has been 10 years. Yeah, or something like that. Almost. Paul Graham once said that Sam is exceedingly good at becoming powerful, which I think is very funny. You are now at the center of the national conversation. I mean, to an extent that I think, you know, sort of has taken us all collectively back. I'm just wondering, how is that for you? Are you like springing out of bed? Or are you like waking up, dreading, you know, the headlines? I saw that you had posted. I don'

In [37]:
from langchain import PromptTemplate


template = """
Read the transcript of the podcast below:
 {podcast_contents}

Summarize the topics discussed in the podcast using bullet points:
"""

podcast_summary_prompt = PromptTemplate(
    input_variables=["podcast_contents"],
    template=template,
)


In [17]:
from langchain import PromptTemplate


template = """
When I listen to a podcast, I take notes on the main talking points of the hosts. I divide it in sections based on topics discussed. 
If the host mentions a specific technology or product, I note that in double brackets like this: [[artificial intelligence]].

These are the notes from the last podcast I listened to:

{podcast_notes}

Write a {words_count} words summary of the notes.
"""

notes_summary_template = PromptTemplate(
    input_variables=["podcast_notes", "words_count"],
    template=template,
)


In [18]:
from langchain import PromptTemplate


template = """
When I take notes for a podcast, I like to also write twitter threads to share them. Each tweet should end saying how far we are in the thread; if it's a 5 tweets thread, the first tweet should end with (1/5), the second one with (2/5), etc.

The tweets have to be easy to read and catch people's attention. Each of them should include an emoji.

These are the notes from my last podcast:

{podcast_notes}

Create a twitter thread for it.
"""

twitter_thread_template = PromptTemplate(
    input_variables=["podcast_notes"],
    template=template,
)


In [46]:
%dotenv

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.text_splitter import CharacterTextSplitter, NLTKTextSplitter, SpacyTextSplitter

llm = OpenAI(model_name="text-davinci-003")
chain = LLMChain(llm=llm, prompt=podcast_summary_prompt)

all_results = []

for podcast in podcasts_to_analyze:
  text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=20)
  text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
  texts = text_splitter.split_text(podcast)
  print(len(texts))
  results = []  
  
  for text in texts:
    subset = chain.run(text)
    results.append(subset)

  all_results.append(results.join('. '))

1


InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 4671 tokens (4415 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [41]:
print("\n".join(results))


-10 years since startup leader's creation
-evaluatement of investments
-focus on VCs
-comment on national conversation around Twitter, combinator

- Sam Alvin, a startup owner, talks about his experience with Hermias, a new company that wants to go at 5 times the speed of sound.
- He is excited about the potential of fusion and solar and wind for Earth and the challenges that they face.
- He also talks about his other investments in the podcast and the importance of sustainability.
- The podcast ends with a discussion of World Coin, a company that Sam Alvin wants to scan your eyeball in exchange for cryptocurrency.

-The company has to re-found its system, and it needs a CEO
-Gary is an excellent CEO
-He has been investing for 20 years and is now leaving the company
-The company needs a CEO, and Gary is a great one
